In [6]:
import torch 
from model import Model
import pandas as pd 
import numpy as np
from torch.utils.data import Dataset,DataLoader

In [7]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
dev = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

In [9]:
class data(Dataset):

    def __init__(self,path):
        self.dataset = np.array(pd.read_csv(path))
        self.map = {'draw': 0, 'erase': 1, 'none': 2}

    def __getitem__(self,index):
        yvals = torch.tensor(self.map[self.dataset[index][0]], dtype=torch.long)
        xvals = torch.from_numpy(self.dataset[index][1:].astype(np.float32))
        return xvals,yvals

    def __len__(self):
        return self.dataset.shape[0]

In [10]:
batch_size = 10
dataset = data('/content/drive/MyDrive/virtual_drawing_board/dataset/dataset.csv')
loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
model = Model().to(dev)
loss_list = []
print(dev)

cuda:0


In [12]:
epochs = 300
lr = 0.000001
crit = torch.nn.CrossEntropyLoss()
opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.8)

for epoch in range(epochs):

    total_loss = 0
    
    for x,y in loader : 
        x = x.to(dev)
        y = y.to(dev)

        opt.zero_grad()
        yhat = model.forward(x)
        loss = crit(yhat,y)
        total_loss += loss
        loss.backward()
        opt.step()

    if epoch % 10 == 0 : 
        print('EPOCH : ', epoch)
        print('  |  Loss : ',total_loss.item(), end='')
        print()
        torch.save(model.state_dict(), f'/content/drive/MyDrive/virtual_drawing_board/models/{epoch}.pt')

    loss_list.append(total_loss.item())

EPOCH :  0
  |  Loss :  8.655169486999512
EPOCH :  10
  |  Loss :  9.393202781677246
EPOCH :  20
  |  Loss :  7.9404072761535645
EPOCH :  30
  |  Loss :  5.851967811584473
EPOCH :  40
  |  Loss :  7.863175868988037
EPOCH :  50
  |  Loss :  5.124624729156494
EPOCH :  60
  |  Loss :  8.510374069213867
EPOCH :  70
  |  Loss :  4.990151405334473
EPOCH :  80
  |  Loss :  6.581913948059082
EPOCH :  90
  |  Loss :  5.790782451629639


KeyboardInterrupt: ignored